#輸入多筆資料至mongodb

In [9]:
import re
import glob
import codecs
import _uniout
import pymongo
from pymongo import MongoClient

def no_court(content):
    court_no = {u"臺北地方":100,u"士林地方":111,u"新北地方":220,u"宜蘭地方":260,u"基隆地方":200,u"桃園地方":320,u"新竹地方":300,u"苗栗地方":350,u"臺中地方":400,u"彰化地方":500,u"南投地方":540,u"臺南地方":700,u"高雄地方":800,u"花蓮地方":970,u"雲林地方":630,u"嘉義地方":602,u"臺東地方":950,u"屏東地方":900,u"澎湖地方":880,u"福建金門":890,u"福建連江":209,u"高雄少年":850}
    m = re.search(u'[  \d]*(灣|福建)(\S{4})法院(民事|家事|暫時|支付)(\S{2})',content)
    return str(court_no[m.group(2)])

def no_jurytype(content):
    jurytype_no = {u"婚":11,u"婚再":12,u"婚更":13,u"家聲":21,u"家訴":22,u"家抗":23,u"家陸許":24,u"司家聲":30,u"基家簡":25,u"重家訴":26,u"婚更一":27,u"婚更(一)":28,u"司家調":29,u"家移調":31,u"家簡":32,u"家調裁":33,u"家查":34,u"家聲抗":35,u"家調裁":36,u"家調補":37,u"司家補":38,u"家親聲":39,u"司家簡調":40,u"家簡上":41,u"家小":42,u"家救":43,u"家小上":44,u"家婚聲":45,u"家再":46,u"家他":47,u"補":90}
    m = re.search(u'(【裁判字號】)(.*?),(.*?),(\d{1,4})[^【】 ]',content)
    return m.group(2)+"_"+str(jurytype_no[m.group(3)])+"_"+m.group(4)

def judgedate(content):
    m = re.search(u'(【裁判日期】)(\d{6,7})',content)
    return m.group(2)

def courtname(content):
    m = re.search(u'[  \d]*(灣|福建)(\S{4})法院(民事|家事|暫時|支付)(\S{2})',content)
    return m.group(2)

def getPK(content):
    return no_court(content)+"_"+no_jurytype(content)+"_"+judgedate(content)


#make a connection to mongodb 
client = pymongo.MongoClient('localhost',27017)
#client = pymongo.MongoClient('mongodb://10.120.28.4',27017)
#assign db_name to db
db = client.test
#make the collection in the db of test
collection = db['dday']
dday = db.dday
#read the judicial case in the directory
for i in glob.glob(u'./samples/*.txt'):
    with codecs.open(i,'r','utf-8') as f :
        content = f.read() # 不能圖超過 2Mb
        f.close() # 在with 中 可不寫,會強制關閉
        #print content
        #CaseNo = re.search(u'(【裁判字號】)(\S{5,14})[^【】 ]',content)
        #make the bson formate and assign to the dic
        dic = {"_id":getPK(content),"juryno":no_jurytype(content),"judgedate":judgedate(content),"courtname":courtname(content)}
        #insert to the collection of dday
        dday_id = dday.insert(dic)


#查詢資料

In [20]:
import _uniout
#print the case number in db.dday
#for i in dday.find({},{'_id':0,'juryno':1,'judgedate':1}):
#for i in dday.find({CaseNo:'/.*/'}):
    #print _uniout.unescape(str(i),'utf-8') 
    #print i.keys()[0]+":"+i.values()[0],i.keys()[1]+":"+i.values()[1]


# 複製collection至新的db

In [21]:
import pymongo
from pymongo import MongoClient
client = pymongo.MongoClient('localhost',27017)
#assign db_name to db
db = client.test
#make the collection in the db of test
collection = db['dday']
dday = db.dday
#create the new db and collection
db2 = client.test2
collection = db2['dday']
dday2 = db2.dday
for i in dday.find():
    insert = dday2.insert(i)
    

# 更新mongodb的文件

In [31]:
import re
import glob
import codecs
import _uniout
import pymongo
from pymongo import MongoClient

def no_court(content):
    court_no = {u"臺北地方":100,u"士林地方":111,u"新北地方":220,u"宜蘭地方":260,u"基隆地方":200,u"桃園地方":320,u"新竹地方":300,u"苗栗地方":350,u"臺中地方":400,u"彰化地方":500,u"南投地方":540,u"臺南地方":700,u"高雄地方":800,u"花蓮地方":970,u"雲林地方":630,u"嘉義地方":602,u"臺東地方":950,u"屏東地方":900,u"澎湖地方":880,u"福建金門":890,u"福建連江":209,u"高雄少年":850}
    m = re.search(u'[  \d]*(灣|福建)(\S{4})法院(民事|家事|暫時|支付)(\S{2})',content)
    return str(court_no[m.group(2)])

def no_jurytype(content):
    jurytype_no = {u"婚":11,u"婚再":12,u"婚更":13,u"家聲":21,u"家訴":22,u"家抗":23,u"家陸許":24,u"司家聲":30,u"基家簡":25,u"重家訴":26,u"婚更一":27,u"婚更(一)":28,u"司家調":29,u"家移調":31,u"家簡":32,u"家調裁":33,u"家查":34,u"家聲抗":35,u"家調裁":36,u"家調補":37,u"司家補":38,u"家親聲":39,u"司家簡調":40,u"家簡上":41,u"家小":42,u"家救":43,u"家小上":44,u"家婚聲":45,u"家再":46,u"家他":47,u"補":90}
    m = re.search(u'(【裁判字號】)(.*?),(.*?),(\d{1,4})[^【】 ]',content)
    return m.group(2)+"_"+str(jurytype_no[m.group(3)])+"_"+m.group(4)

def judgedate(content):
    m = re.search(u'(【裁判日期】)(\d{6,7})',content)
    return m.group(2)

def getPK(content):
    return no_court(content)+"_"+no_jurytype(content)+"_"+judgedate(content)

#make a connection to mongodb 
client = pymongo.MongoClient('localhost',27017)
#assign db_name to db
db = client.test
#make the collection in the db of test
collection = db['dday']
dday = db.dday
#read the judicial case in the directory
dir_path = glob.glob(u'./samples/*.txt')
for i in dir_path:
    with codecs.open(i,'r','utf-8') as f :
        content = f.read() # 不能圖超過 2Mb
        f.close() # 在with 中 可不寫,會強制關閉
        #create the new field
        filename = re.search(u'(.*)/(.*?).txt',i).group(2)
        #update new field to the dday collection
        update = dday.update({"_id":getPK(content)},{"$set":{"filename":filename}},upsert=False,)

        

#刪除collection

In [8]:
import pymongo
from pymongo import MongoClient
#make a connection to mongodb 
client = pymongo.MongoClient('localhost',27017)
#client = pymongo.MongoClient('mongodb://10.120.28.4',27017)
#account of db ??
db = client.test
#make the collection in the db of test
collection = db['dday']
dday = db.dday
#Drop the collection
db.drop_collection('dday')
